In [11]:
import PySimpleGUI as sg
import pandas as pd
import numpy as np
import wikipedia
import pyttsx3
import matplotlib.pyplot as plt
import sqlite3
import time
import pygame
from PIL import Image
from reportlab.pdfgen import canvas
from reportlab.platypus import SimpleDocTemplate
from reportlab.lib import pdfencrypt
from reportlab.lib.units import inch
from reportlab.platypus.tables import Table, TableStyle 
from reportlab.lib.pagesizes import letter

from reportlab.lib import colors 
from reportlab.lib.enums import TA_LEFT, TA_CENTER, TA_RIGHT
import datetime

%matplotlib inline

######################################CONNECTION WITH SQL DATABASE#####################################################
df=pd.read_excel("Quiz_Qestion.xlsx")
#print(df)

conn = sqlite3.connect('Users.db')
c = conn.cursor()

c.execute("""CREATE TABLE IF NOT EXISTS user (
        Username VARCHAR(20) NOT NULL,
        First_Name VARCHAR(20) NOT NULL,
        Last_Name VARCHAR(20) NOT NULL,
        Email VARCHAR(20) NOT NULL,
        Password VARCHAR(20) NOT NULL
        )""")
conn.commit()

"""
########################################CODE FOR SPEAK RESULTS#########################################################  
engine=pyttsx3.init("sapi5")
voices=engine.getProperty("voices")
#print(voices[1].id)
engine.setProperty("voice",voices[0].id)



def speak(audio):
    engine.say(audio)
    engine.runAndWait()

"""
display_name=""

"""
#####################################FUNCTION FOR ADDIND NEW USER######################################################
def new_user():
    with conn:
        insert='INSERT INTO user VALUES (?,?,?,?,?)'
        c.execute(insert,[values["name"],values["first_name"],values["last_name"],values["email"],values["password"]])
        conn.commit()
        r=c.fetchall()
        if r:
            display_name=r[0]
            for i in r:
                print("Welcome: "+i[1]+" ,you have registered")
                display_name=i[1]
                window1.Close()
                break
####################################FUNCTION FOR LOGIN USER############################################################
def login():
    with conn:
        find_user=("SELECT * FROM User WHERE Username=? AND Email=? AND Password=?")
        c.execute(find_user,([(name),(email),(password)]))
        results=c.fetchall()

        if results:
            for i in results:
                print("Welcome: "+i[1])
                display_name=i[1]
                print(i[1],type(i[1]))
                print("Welcome:"+i[1])
                break
        else:
            print("Username and password not recognised")
            again=input("Do you want to try again?y/n ")
            if again.lower()=="n":
                print("Goodbye")
                #speak("Goodbye")
                time.sleep(.1)
            elif again.lower()=="y":
                print("Please enter username and password ")
                time.sleep(.1)
        window2.Close()

###################CODE FOR SELECTING REGISTRATION WINDOW OR LOGIN WINDOW#############################################
layout=[
        [sg.Text(" ",size=(13,7)),sg.Image(r'1.png')],
        [sg.Text("Please Select Appropriate Option to Start Quiz",size=(25,2),font="Helvetica 30",justification='center')],
        [sg.Button("Register_for_Quiz"), sg.Button("Login_in_Quiz")]
       ]
window= sg.Window('SELECTING_REGISTRATION_WINDOW_OR_LOGIN_WINDOW').Layout(layout)
while True:
    event,values=window.Read()
    ###########################################CODE FOR REGISTRATION WINDOW############################################
    if(event=="Register_for_Quiz"):
        window.Close()
        layout1=[
                [sg.Text("User Registration",size=(20,3),font="Courier 15")],
                [sg.Text('User Name', size=(15, 1)), sg.InputText('',key="name")],
                [sg.Text('First_Name', size=(15, 1)), sg.InputText('',key="first_name")],
                [sg.Text('Last_Name', size=(15, 1)), sg.InputText('',key="last_name")],
                [sg.Text('Email', size=(15, 1)), sg.InputText('',key="email")],
                [sg.Text('Password', size=(15, 1)), sg.InputText('',key="password")],
                [sg.Button("Submit"), sg.Button("Exit")]
        ]

        window1= sg.Window('Registration_For_Quiz_Application').Layout(layout1)
        #speak("You had click on Register_for_Quiz Button to Register, Now enter User Name,First_Name,Last_Name,Email,Password")

        while True:
            event,values=window1.Read()
            print("event= ", event,"values=",values)
            if(event=="Submit"):
                new_user()        
            elif(event is None or event=="Exit"):
                window1.Close()
                break 
            break    
    ####################################CODE FOR LOGIN WINDOW##############################################################
    elif(event=="Login_in_Quiz"):
        window.Close()
        layout2=[
                [sg.Text("User Login",size=(15,3),font="Courier 15")],
                [sg.Text('User Name', size=(15, 1)), sg.InputText('',key="name1")],
                [sg.Text('Email', size=(15, 1)), sg.InputText('',key="email1")],
                [sg.Text('Password', size=(15, 1)), sg.InputText('',key="password1")],
                [sg.Button("Login"), sg.Button("Exit")]
        ]

        window2= sg.Window('Login_For_Quiz_Application').Layout(layout2)
        #speak("You had click on Login_in_Quiz Button to Login, Now enter User Name,Email,Password")
        while True:
            event2,values2=window2.Read()
            print("event= ", event2,"values=",values2)
            #Start_window.Close()
            name=values2["name1"]
            email=values2["email1"]
            password=values2["password1"]
            if(event2=="Login"):
                login()                
            elif(event is None or event=="Exit"):
                window2.Close()
                break
            break
        break  
"""
########################################CODE FOR TIMER SOUND###########################################################    
def Sound():
    pygame.mixer.init()
    pygame.mixer.music.load("tictiksound.mp3")
    pygame.mixer.music.play()

########################################CODE FOR TIMER SOUND###########################################################
def Alarm():
    pygame.mixer.init()
    pygame.mixer.music.load("LoudAlarm.mp3")
    pygame.mixer.music.play()
#########################################CODE FOR TIMER################################################################    
def time_as_int():
    return int(round(time.time() * 100))


########################################CODE FOR GENERATION OF QUIZ####################################################       

list=[]
l1=[]     
list_Y=[]    
Score=0
r=[]

r = np.random.choice(29, 10 , replace=False)
#r=numbers

'''
for i in range(10):
    n=np.random.randint(0,30)
    r.append(n)
'''

#print(r)
j=0
k=0
amount=30
layout4=[
            [sg.Text("Welcome: ",size=(15, 2),font=('Helvetica', 20),text_color='red'),
             sg.Text(display_name,size=(15, 2),font=('Helvetica', 20),text_color='blue')],
            [sg.Text(str(df.iloc[r[j],1]),key="Question",size=(80, 2))],
            [sg.Radio("","Radio",key="Radio1"),
             sg.Text(str(df.iloc[r[j],2]),key="tRadio1",size=(40, 1)),
             sg.Text("Time Remaining: ",size=(15, 2),font=('Helvetica', 20), justification='center')],
            [sg.Radio("","Radio",key="Radio2"),
             sg.Text(str(df.iloc[r[j],3]),key="tRadio2",size=(40, 1)),
             sg.Text(amount, size=(15, 2),font=('Helvetica', 20), justification='center', key="_timer")],
            [sg.Radio("","Radio",key="Radio3"),
             sg.Text(str(df.iloc[r[j],4]),key="tRadio3",size=(40, 1)),
             sg.Text("Score: ",size=(15, 2),font=('Helvetica', 20), justification='center')],
            [sg.Radio("","Radio",key="Radio4"),
             sg.Text(str(df.iloc[r[j],5]),key="tRadio4",size=(40, 1)),
             sg.Text(Score,size=(15, 2),font=('Helvetica', 20), justification='center',key="_score")],
            [sg.Button("Next"),sg.Button("Lock_Answer"),sg.Button("Quit")]
        ]
window4= sg.Window('Quiz_Application').Layout(layout4)
start_time=time_as_int()
while(j<9):
    print("j= ",j)
    event, values = window4.Read(1000) # run every 10 milliseconds
    k+=1
    #sg.PopupAnimated(sg.DEFAULT_BASE64_LOADING_GIF,no_titlebar=False,location=(850,480))
    current_time = time_as_int() - start_time
    v=30-(((current_time // 100) % 60)%31)
    window4.FindElement('_timer').Update('{:02d}'.format(v))
    
    Sound()
    l=values
    
    #event, values = window4.Read(timeout=1000) # run every 1 Second
    if amount:
        
        window4.FindElement('_timer').Update('{:02d}'.format(amount))
        amount -= 1
    k+=1
    
    Sound()
    #Alarm()
    l=values
    
    if(event is None or event=="Quit"):
        #sg.PopupAnimated(image_source=None)
        list_Y.append(0)
        break

    elif(event=="__TIMEOUT__"):
        if(k==9):
            window4.FindElement('_timer').Update('Times up')
            amount=30
            list_Y.append(0)
            #sg.PopupAnimated(image_source=None)
            j+=1
            window4.FindElement('Question').Update(str(df.iloc[r[j],1]))
            window4.FindElement('tRadio1').Update(str(df.iloc[r[j],2]))
            window4.FindElement('tRadio2').Update(str(df.iloc[r[j],3]))
            window4.FindElement('tRadio3').Update(str(df.iloc[r[j],4]))
            window4.FindElement('tRadio4').Update(str(df.iloc[r[j],5]))
            window4.FindElement('_score').Update(Score)
            k=0
            
    elif(event=="Lock_Answer"):
        amount=30
        #sg.PopupAnimated(image_source=None)
        if(l["Radio1"]==True):
            if(df.iloc[r[j],2]==df.iloc[r[j],6]):
                #print("Your Answer is correct")
                sg.PopupTimed("Correct Answer",auto_close=True,
                              auto_close_duration=2,non_blocking=True,
                              no_titlebar=False,keep_on_top=True)
                Score+=10
                list_Y.append(Score)
                #window4.Close()
            else:
                sg.PopupTimed("Wrong Answer",auto_close=True,
                              auto_close_duration=2,non_blocking=True,
                              no_titlebar=False,keep_on_top=True)
                list_Y.append(0)
                
        elif(l["Radio2"]==True):
            if(df.iloc[r[j],3]==df.iloc[r[j],6]):
                sg.PopupTimed("Correct Answer",auto_close=True,
                              auto_close_duration=2,non_blocking=True,
                              no_titlebar=False,keep_on_top=True)
                Score+=10
                list_Y.append(Score)
            else:
                sg.PopupTimed("Wrong Answer",auto_close=True,
                              auto_close_duration=2,non_blocking=True,
                              no_titlebar=False,keep_on_top=True)
                list_Y.append(0)

        elif(l["Radio3"]==True):
            if(df.iloc[r[j],4]==df.iloc[r[j],6]):
                sg.PopupTimed("Correct Answer",auto_close=True,
                              auto_close_duration=2,non_blocking=True,
                              no_titlebar=False,keep_on_top=True)
                Score+=10
                list_Y.append(Score)
            else:
                sg.PopupTimed("Wrong Answer",auto_close=True,
                              auto_close_duration=2,non_blocking=True,
                              no_titlebar=False,keep_on_top=True)
                list_Y.append(0)
                
        elif(l["Radio4"]==True):
            if(df.iloc[r[j],5]==df.iloc[r[j],6]):
                sg.PopupTimed("Correct Answer",auto_close=True,
                              auto_close_duration=2,non_blocking=True,
                              no_titlebar=False,keep_on_top=True)
                Score+=10
                list_Y.append(Score)
            else:
                sg.PopupTimed("Wrong Answer",auto_close=True,
                              auto_close_duration=2,non_blocking=True,
                              no_titlebar=False,keep_on_top=True)
                list_Y.append(0)
        else:
            #print("Your Answer is Wrong")
            sg.PopupTimed("Nothing was selected",auto_close=True,
                              auto_close_duration=2,non_blocking=True,
                              no_titlebar=False,keep_on_top=True)
            #print(str(df.iloc[r[j],6]))
        try:
            summary2=wikipedia.summary(str(df.iloc[r[j],6]),sentences=5)
            '''
            sg.PopupTimed("Correct Answer is: {}".format(summary2),auto_close=True,
                              auto_close_duration=3,non_blocking=True,
                              no_titlebar=False,keep_on_top=True)
            '''
            #speak(summary2)
        except:
            print("No Page Found on WIKIPEDIA")
            #speak("No Page Found on WIKIPEDIA")
            #summary=wikipedia.summary(str(df.iloc[r,1]),sentences=5)
            #print(str(df.iloc[r,1]))
            #print(summary)
            #speak(summary)
        j+=1
        window4.FindElement('Question').Update(str(df.iloc[r[j],1]))
        window4.FindElement('tRadio1').Update(str(df.iloc[r[j],2]))
        window4.FindElement('tRadio2').Update(str(df.iloc[r[j],3]))
        window4.FindElement('tRadio3').Update(str(df.iloc[r[j],4]))
        window4.FindElement('tRadio4').Update(str(df.iloc[r[j],5]))
        window4.FindElement('_score').Update(Score)
        k=0
        
    elif(event=="Next"):
        amount=30
        list_Y.append(0)
        #sg.PopupAnimated(image_source=None)
        j+=1
        window4.FindElement('Question').Update(str(df.iloc[r[j],1]))
        window4.FindElement('tRadio1').Update(str(df.iloc[r[j],2]))
        window4.FindElement('tRadio2').Update(str(df.iloc[r[j],3]))
        window4.FindElement('tRadio3').Update(str(df.iloc[r[j],4]))
        window4.FindElement('tRadio4').Update(str(df.iloc[r[j],5]))
        window4.FindElement('_score').Update(Score)
        k=0
        continue
    window4.Close()
    #print("display_name = ",display_name)        

#########################################CODE FOR SUBMIT WINDOW########################################################
x=np.arange(1,11,1)
#print(x)
layout5=[[sg.Button("Submit")]]
window5 = sg.Window('Quiz_Application_Submit').Layout(layout5)
event,values=window5.Read()
#print("event= ", event,"values=",values)
if(event=="Submit"):
    print("Score=",Score)
x=np.arange(1,10,1)
#print("x =",x)
y=list_Y
#print("y= ",y)
plt.title("Performance Chart")
plt.xlabel("Question No.")
plt.ylabel("Score")
plt.plot(x,y)
plt.savefig("Figure.png")
plt.show()
window5.Close()

######################################CODE FOR DISPLAY TIME############################################################
time=time.time()
print(time)
days=time//(24*3600)
time=time%(24*3600)
hour=int(time//3600)
time=time%3600
minutes=int(time//60)
time=time//60
seconds=int(time)
current_time=str(hour)+":"+str(minutes)+":"+str(seconds)
print(current_time)

######################################CODE FOR FLIP THE QUESTION#######################################################
im= Image.open("Figure.png")
out=im.transpose(Image.FLIP_LEFT_RIGHT)
ino=out.transpose(Image.FLIP_TOP_BOTTOM)
ino=ino.transpose(Image.FLIP_LEFT_RIGHT)
ino.save("Fig1.png")
ino.show()

######################################CODE FOR GENERATION OF PDF#######################################################
enc=pdfencrypt.StandardEncryption("1234",canPrint=1)
pdf=canvas.Canvas("Project_demo.pdf",bottomup=0,encrypt=enc)
pdf.translate(inch,inch)
fonts=pdf.getAvailableFonts()
print(fonts)
############################################ Heading ##################################################################
pdf.setFont("Courier-BoldOblique",40)
pdf.setFillColor("orange")
pdf.drawCentredString(250,50,"Report For Quiz")
################# Username,EmailId,Date,Time ##########################################################################
pdf.setFont("Courier-Bold",20)
pdf.setFillColor("cyan")
pdf.drawString(0,100,"Username:"+values2["name1"])
pdf.setFillColor("cornflower")
pdf.drawString(0,130,"Email_ID:"+values2["email1"])
pdf.setFillColor("thistle")
pdf.drawString(330,100,"Date:"+str(datetime.date.today()))
pdf.setFillColor("cadetblue")
pdf.drawString(330,130,"Time:"+str(current_time))



pdf.drawImage("Fig1.png",100,160,300,350)
pdf.save()

j=  0
j=  0


TclError: bad window path name ".!toplevel17"